In [14]:
import tensorflow as tf
import numpy as np
import scipy

import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
import glob, os, sys, gc

from prior_attention import build_prior_attentions
from utils import Utils
from config import Config

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM
from keras import optimizers
from keras.optimizers import Adam

# this is important to make sure the Config is reloaded when modified
%load_ext autoreload
%autoreload 2
%pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Pretty printing has been turned ON


In [15]:
dict(vars(Config))

{'__module__': 'config',
 'main_path': '/home/mikey/Data/POETdataset/PascalImages',
 'class_names': ['dog',
  'aeroplane',
  'boat',
  'bicycle',
  'cat',
  'cow',
  'diningtable',
  'horse',
  'motorbike',
  'sofa'],
 'IMG_SIZE': 480,
 'PATCH_WIDTH': 60,
 'SMALLER_IMG_SIZE': 150,
 'T': 64,
 'new_dir': 'soft_attention_features',
 'new_dir_img': 'soft_attention_images',
 'name_to_class_dict': {'dog': 0,
  'aeroplane': 1,
  'boat': 2,
  'bicycle': 3,
  'cat': 4,
  'cow': 5,
  'diningtable': 6,
  'horse': 7,
  'motorbike': 8,
  'sofa': 9},
 'train_ids': ['dog_2010_002398.jpg',
  'horse_2009_005161.jpg',
  'cat_2010_002662.jpg',
  'cat_2010_003687.jpg',
  'aeroplane_2008_006677.jpg',
  'cat_2010_000389.jpg',
  'bicycle_2009_003187.jpg',
  'motorbike_2010_005366.jpg',
  'dog_2010_000276.jpg',
  'dog_2010_005330.jpg',
  'aeroplane_2008_005905.jpg',
  'aeroplane_2008_008179.jpg',
  'cat_2008_003772.jpg',
  'cat_2009_004477.jpg',
  'motorbike_2011_000397.jpg',
  'cat_2010_001939.jpg',
  'bicyc

In [16]:
main_path = Config.main_path
class_names = Config.class_names
IMG_SIZE = Config.IMG_SIZE
PATCH_WIDTH = Config.PATCH_WIDTH
SMALLER_IMG_SIZE = Config.SMALLER_IMG_SIZE
T = Config.T
new_dir = Config.new_dir
new_dir_img = Config.new_dir_img
name_to_class_dict = Config.name_to_class_dict
train_ids, test_ids = Config.train_ids, Config.test_ids

In [17]:
def load_data(mode):
    X = []
    y = []
    org_images = []
    all_ids = []
    for class_name, class_number in name_to_class_dict.items():
        classfeatures_path = os.path.abspath(os.path.join('.', new_dir, class_name + '_' + mode + '_features.pkl'))
        classimages_path = os.path.abspath(os.path.join('.', new_dir_img, class_name + '_' + mode + '.pkl'))
        print(classfeatures_path)

        features = Utils.load_object(classfeatures_path).reshape(-1,T,2048)
        images = Utils.load_object(classimages_path)
        ids = Utils.load_object(new_dir + os.path.sep + class_name + '_'+mode+'_names.pkl')
        org_images.append(images)

        print(features.shape)
        X.append(features)
        y.append([class_number] * features.shape[0])
        all_ids.append(ids)

    X = np.concatenate(X)
    y = np.concatenate(y)
    y = keras.utils.to_categorical(y)
    ids = np.concatenate(all_ids)
    org_images = np.concatenate(org_images)
    return X, y, org_images, ids
    
X_train, y_train, images_train, train_ids = load_data(mode='train')
X_test, y_test, images_test, test_ids = load_data(mode='test')

X_train.shape, y_train.shape, images_train.shape, X_test.shape, y_test.shape, images_test.shape

/home/mikey/Repos/visual-attention-cnn-and-eye-tracking/soft_attention_model/soft_attention_features/dog_train_features.pkl
(1128, 64, 2048)
/home/mikey/Repos/visual-attention-cnn-and-eye-tracking/soft_attention_model/soft_attention_features/aeroplane_train_features.pkl
(599, 64, 2048)
/home/mikey/Repos/visual-attention-cnn-and-eye-tracking/soft_attention_model/soft_attention_features/boat_train_features.pkl
(454, 64, 2048)
/home/mikey/Repos/visual-attention-cnn-and-eye-tracking/soft_attention_model/soft_attention_features/bicycle_train_features.pkl
(480, 64, 2048)
/home/mikey/Repos/visual-attention-cnn-and-eye-tracking/soft_attention_model/soft_attention_features/cat_train_features.pkl
(943, 64, 2048)
/home/mikey/Repos/visual-attention-cnn-and-eye-tracking/soft_attention_model/soft_attention_features/cow_train_features.pkl
(265, 64, 2048)
/home/mikey/Repos/visual-attention-cnn-and-eye-tracking/soft_attention_model/soft_attention_features/diningtable_train_features.pkl
(427, 64, 2048)


((5517, 64, 2048),
 (5517, 10),
 (5517, 150, 150, 3),
 (614, 64, 2048),
 (614, 10),
 (614, 150, 150, 3))

In [ ]:
tf.reset_default_graph()

n_epochs = 100
batch_size = 32
T = X_train.shape[1]
lstm_time = 3 #analogous to fixations ?
regularize_attention = False
pretrain_attention = True

prior_attentions = build_prior_attentions(Utils.load_object('./train_fix.pkl'), T, lstm_time, gauss_sigma=None)
               
X_input = tf.placeholder(tf.float32, [None, T, 2048], name='X')
prior_attentions_input = tf.placeholder(tf.float32, [None, lstm_time, T])
labels = tf.placeholder(tf.int32, [None, y_train.shape[1]], name='labels')

rnn_units = 128

c,h = Utils._get_initial_lstm(X_input[:,:,:], H=rnn_units)

context = tf.layers.dense(tf.reshape(X_input, shape=(-1, T*2048)), rnn_units, activation='tanh')
# context = tf.layers.dropout(context, rate=0.1)

lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=rnn_units)

contexts = []
attentions = []

att_input = tf.nn.tanh(X_input) # squeeze features from Resnet into same scale as context

# att_input = tf.layers.dense(tf.reshape(X_input, shape=(-1, T*2048)), T, activation='tanh')
# AttentionDense = tf.layers.Dense(T, activation='tanh')

kernel_reg, activity_reg = None, None
if regularize_attention:
    kernel_reg, activity_reg = (tf.contrib.layers.l2_regularizer(scale=.0005),
                                tf.contrib.layers.l2_regularizer(scale=.0005))

ContextDense = tf.layers.Dense(2048, activation='tanh',
                              kernel_regularizer=kernel_reg,
                              activity_regularizer=activity_reg)
for t in range(lstm_time):
    # encoded lstm context to a specific size 
    context = ContextDense(context)
    context = tf.layers.dropout(context, rate=0.1)
    
    # how lstm context and patch info relate
    att = tf.einsum('bi,bti->bt', context, att_input)
#     att = AttentionDense(tf.concat([context, att_input], axis=1))
    att = tf.nn.softmax(att)
#     attention_loss = 
    
    lstm_input = tf.einsum('bt,bti->bi', att, X_input)
    
    context, (c, h) = lstm_cell(inputs=lstm_input, state=[c, h])
#     context = tf.layers.dropout(context, rate=0.2)
    contexts.append(context)
    attentions.append(att)
    
all_attentions_tensor = tf.transpose(tf.stack(attentions), [1,0,2])
attention_loss = tf.reduce_mean(tf.abs(all_attentions_tensor - tf.nn.softmax(prior_attentions_input)))

Dense = tf.layers.Dense(units=32, activation=tf.nn.relu)
Logits = tf.layers.Dense(units=y_train.shape[1])
probabilities_list = []
for context in contexts:
    dense = Dense(context)
    dense = tf.layers.dropout(dense, rate=0.2)

    # Logits Layer
    logits = Logits(dense)
    logits = tf.layers.dropout(logits, rate=0.2)

    classes = tf.argmax(logits, axis=1)
    probabilites = tf.nn.softmax(logits)
    probabilities_list.append(probabilites)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logits))
l2_loss = tf.losses.get_regularization_loss()
loss += l2_loss

regular_loss_multiplier = tf.placeholder_with_default(1.0, shape=(), name='regular_loss_multiplier')
attention_loss_multiplier = tf.placeholder_with_default(0., shape=(), name='attention_loss_multiplier')

final_loss = regular_loss_multiplier * loss + attention_loss_multiplier * attention_loss

learning_rate = tf.placeholder(tf.float32, name='learning_rate')

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(
    loss=final_loss,
    global_step=tf.train.get_global_step()
)

accuracy = tf.reduce_mean(tf.cast(tf.equal(classes, tf.argmax(labels, 1)), tf.float32))

init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

Crucial points for high accuracy:
- regularization on attention
- shuffle after every epoch
- lstm time doesn't matter much (run experiments)
- some dropout to make sure it isn't stuck in local minima

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    learning_rate_ = 0.0001
    changed_learning_rate = False
    
    training_attention = pretrain_attention
    for epoch in range(n_epochs):
        if training_attention:
            att_loss_multiplier_value = 1.
            regular_loss_multiplier_value = 0.
        else:
            att_loss_multiplier_value = 10.
            regular_loss_multiplier_value += .1
            if regular_loss_multiplier_value > 1:
                regular_loss_multiplier_value = 1.
                att_loss_multiplier_value = 0.
            
        print("Epoch:", epoch)
        train_acc = []
        att_losses = []
        for b in range(0, X_train.shape[0], batch_size):
            _, acc, loss_, att_loss_ = sess.run([train_op, accuracy, final_loss, attention_loss], feed_dict={
                                                            X_input: X_train[b:b+batch_size],
                                                            prior_attentions_input: prior_attentions[b:b+batch_size],
                                                            labels: y_train[b:b+batch_size],
                                                            learning_rate: learning_rate_,
                                                            attention_loss_multiplier: att_loss_multiplier_value,
                                                            regular_loss_multiplier: regular_loss_multiplier_value
                                                        })
            train_acc.append(acc)
            att_losses.append(att_loss_)
            
        if training_attention and np.mean(att_losses) < 0.004:
            training_attention = False
            print("Done training prior attention")
            break
            
        # reshuffle after every epoch
        if not training_attention:
            X_train, y_train, prior_attentions, images_train = \
                Utils.unison_shuffled_copies([X_train, y_train, prior_attentions, images_train])
            
            acc, context_, atts_, test_proba_, probabilities_list_ = \
                sess.run([accuracy,
                          context, attentions, probabilites,
                          probabilities_list], 
                         feed_dict={
                                    X_input: X_test,
                                    labels: y_test
                                    })
            if acc > 0.8 and not changed_learning_rate:
                learning_rate_ = learning_rate_ / 10.
                changed_learning_rate = True
                print("Adjusted learning rate to", learning_rate_)

            if acc > 0.97:
                break
                     
        print("Train:", np.mean(train_acc))
        print("Test:", acc)
        print("Attention loss", np.mean(att_losses))
        train_acc = []
        att_losses = []
#         print(atts_[0])

In [ ]:
np.array(context_).shape, np.array(atts_).shape

In [ ]:
atts_ = np.array(atts_)
images_test = np.array(images_test)
probabilities_list_ = np.array(probabilities_list_)

In [ ]:
unique, counts = np.unique(np.argmax(y_test, axis=-1), return_counts=True)
class_to_name_dict = {v:k for k,v in name_to_class_dict.items()}
unique = np.array([class_to_name_dict[u] for u in unique])

np.asarray((unique, counts)).T

In [ ]:
unique, counts = np.unique(np.argmax(test_proba_, axis=-1), return_counts=True)
class_to_name_dict = {v:k for k,v in name_to_class_dict.items()}
unique = np.array([class_to_name_dict[u] for u in unique])

np.asarray((unique, counts)).T

In [ ]:
test_fix = Utils.load_object('./test_fix.pkl')
test_ids = Utils.load_object('./test_ids.pkl')

name_to_fix_test = {name:fix for fix, name in zip(test_fix, test_ids)}

In [ ]:
from scipy import signal
from scipy import ndimage

class_name = 'aeroplane'
class_id = name_to_class_dict[class_name]
class_mask = np.argmax(y_test, axis=1) == class_id
class_images = images_test[class_mask]
class_test_proba = test_proba_[class_mask]
class_proba_list = probabilities_list_[:,class_mask,:]
class_atts = atts_[:,class_mask,:]
print(class_proba_list.shape)
image_n = 5
rescale = IMG_SIZE / SMALLER_IMG_SIZE
image_name = np.array(test_ids)[class_mask][image_n]

print(np.array(test_ids)[class_mask][image_n])
print(name_to_fix_test[image_name])

for t in range(atts_.shape[0]):
    image = class_images[image_n]
    proba_res = class_proba_list[t,image_n,:]
#     print(proba_res)
#     print(class_to_name_dict[np.argmax(proba_res)])
    proba_res = [(i, proba) for i, proba in enumerate(proba_res)]
    proba_res.sort(key= lambda pair: pair[1], reverse=True)
#     print(proba_res)
    for i, proba in enumerate(proba_res):
        print(class_to_name_dict[proba[0]], proba[1])
    
    
    if rescale != 1:
        chosen_img = np.array(Utils.normalize(skimage.transform.rescale(image.copy(), rescale))) * 500
    else:
        chosen_img = image.copy()
    print(chosen_img.shape)
        
    chosen_attention = class_atts[t,image_n,:]
    
    result = np.zeros_like(chosen_img)
    for attended_patch_n in range(T):
        att_prob = chosen_attention[attended_patch_n]
#         print(att_prob)
#         print(attended_patch)
#         print(attended_patch_n/(IMG_SIZE/PATCH_WIDTH))
#         print(int(attended_patch%(IMG_SIZE/PATCH_WIDTH)))
        row = int(attended_patch_n/(IMG_SIZE/PATCH_WIDTH))
        col = int(attended_patch_n%(IMG_SIZE/PATCH_WIDTH))
#         print(attended_patch_n, row, col)
        from_y = row*PATCH_WIDTH
        to_y = from_y+PATCH_WIDTH

        from_x = col*PATCH_WIDTH
        to_x = from_x+PATCH_WIDTH
#         print(from_x, from_y)
        result[from_y:to_y, from_x:to_x, :] = np.ones_like(result[from_x:to_x,from_y:to_y,:]) * 255 * att_prob

    
    result = result / 255.
    chosen_img = chosen_img / 255.
    result = ndimage.gaussian_filter(result, sigma=32) * 8
    result[result > 1] = 1
    res = Utils.normalize(0.3 * chosen_img + result)
    plt.imshow(res)
    plt.show()
    
    plt.bar(range(atts_.shape[2]), chosen_attention)
    plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#     print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    return plt.gcf()

from sklearn.metrics import confusion_matrix

y_pred_proba = test_proba_

y_pred = np.argmax(y_pred_proba, axis=1)

y_test_argmax = np.argmax(y_test, axis=1)

y_test_argmax.shape

cnf_matrix = confusion_matrix(y_test_argmax, y_pred)

import itertools

plt.figure(figsize=(12,12))
fig = plot_confusion_matrix(cnf_matrix, class_names, normalize=True)

In [ ]:
# files_list = [glob.glob(os.path.join(main_path, '%s*' %class_)) for class_ in class_names]

# ids_ = []
# final_files_list = []
# labels = []
# for label, files in enumerate(files_list):
#     one_list = []
#     for file in files:
#         id_ = ''.join(file.split('/')[-1].split('.')[0].split('_')[1:])
#         if id_ not in ids_:
#             one_list.append(file)
#             ids_.append(id_)
#             labels.append(label)
#     final_files_list.append(one_list)
    

# files_dict = {class_name.replace('*', ''): class_files for class_name, class_files in zip(class_names, final_files_list)}